In [1]:
import pandas as pd
import re
from tqdm import tqdm
import math
import numpy as np
import json
from ast import literal_eval

In [3]:
# All QSW records.
df_qsw_raw=pd.read_excel('../data_raw/quan_song_wen_muzhi.xlsx', sheet_name='墓誌銘墓表壙誌行狀神道碑塔銘墓碑')[['content_id', 'content', 'subject', 'author']].set_index('content_id')
df_qsw_raw['content']=[str(s) for s in df_qsw_raw['content']]
df_qsw_raw.dropna(inplace=True)
df_qsw_raw = df_qsw_raw[df_qsw_raw['content'] != "nan"]

/usr/remote/anaconda-3.7-deeplearn/lib/python3.7/site-packages/openpyxl/worksheet/_reader.py:300: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [4]:
tiangan = ['甲子','乙丑','丙寅','丁卯','戊辰','己巳','庚午','辛未','壬申','癸酉','甲戌','乙亥','丙子','丁丑','戊寅','己卯','庚辰','辛巳','壬午','癸未','甲申','乙酉','丙戌','丁亥',
'戊子','己丑','庚寅','辛卯','壬辰','癸巳','甲午','乙未','丙申','丁酉','戊戌','己亥','庚子','辛丑','壬寅','癸卯','甲辰','乙巳','丙午','丁未','戊申','己酉','庚戌','辛亥',
'壬子','癸丑','甲寅','乙卯','丙辰','丁巳','戊午','己未','庚申','辛酉','壬戌','癸亥']

In [5]:
string = ""
for t in tiangan:
    string = string + t + "|"
string = string[:-1]

In [6]:
string = "(" + string + ")"
string

'(甲子|乙丑|丙寅|丁卯|戊辰|己巳|庚午|辛未|壬申|癸酉|甲戌|乙亥|丙子|丁丑|戊寅|己卯|庚辰|辛巳|壬午|癸未|甲申|乙酉|丙戌|丁亥|戊子|己丑|庚寅|辛卯|壬辰|癸巳|甲午|乙未|丙申|丁酉|戊戌|己亥|庚子|辛丑|壬寅|癸卯|甲辰|乙巳|丙午|丁未|戊申|己酉|庚戌|辛亥|壬子|癸丑|甲寅|乙卯|丙辰|丁巳|戊午|己未|庚申|辛酉|壬戌|癸亥)'

In [9]:
# Function for seperating sentences.
def sep_mark_sent(string):
    string=string.replace('，', '/ws1ep/').replace('；', '/ws2ep/').replace('、', '/ws3ep/')
    string=string.replace('！', '/wend/').replace('。', '/wend/').replace('？', '/wend/').replace('！', '/wend/')
    string=string.replace('：', '/wm/')
    return [s for s in string.split("/wend/") if s!='']

In [66]:
# Seperate sentences and retain sentences with kinship words.
all_sent_count=0
kin_sent_count=0
df=pd.DataFrame()
for index in tqdm(df_qsw_raw.index):
    content=df_qsw_raw.loc[index]['content']
    sent_list=sep_mark_sent(content) # Seperate sentences.
    all_sent_count+=len(sent_list)
    subject=df_qsw_raw.loc[index]['subject']
    author=df_qsw_raw.loc[index]['author']
    kin_sent_list=[]
    content_id=index
    for sent in sent_list:
        if "春秋" in sent or "享年" in sent or "得年" in sent:
            sent = sent + "/"
            kin_sent_list.append(sent)
    df=pd.concat([pd.DataFrame(data=[[content_id, subject, author, sent, content] for sent in kin_sent_list],
                                          columns=['content_id', 'subject', 'author', 'sent', 'content']
                                          ), df], axis=0, ignore_index=True
                            )

100%|██████████| 4701/4701 [00:05<00:00, 929.00it/s]


In [77]:
re_rule = "((得年|享年|春秋)((一|二|三|四|五|六|七|八|九|十|有)*?))\/"
for index in tqdm(df.index):
    row = df.loc[index]
    find = re.findall(re_rule, row['sent'])
    if find:         
        df.loc[index, 'length'] = find[0][0]

100%|██████████| 2963/2963 [00:00<00:00, 4711.62it/s]


In [78]:
df_length = df.dropna()

In [82]:
df_length

,content_id,subject,author,sent,content,length
0,8715497.0,申之,杜士熊,間嬰疾/ws1ep/卒於正寢/ws1ep/時嘉定八年四月十二日也/ws1ep/享年四十有一/,〔額文〕宋故都監杜公墓碑〔碑文〕先君姓杜，諱申之，字伯禄，世居嘉興府華亭之章廟。曾祖顯，故不...,享年四十有一
1,5958020.0,徐洪,鄧光薦,殁於淳祐乙巳五月乙卯/ws1ep/享年七十一/,頃予登句曲，直東南望，數峰儼然，水倉玉立。其下谿谷迴合，林隰衍沃，其氣乍清乍潤，類非山澤枯槁...,享年七十一
2,5858013.0,姚懋,姚珤,二十六年/ws1ep/改授沙縣浮流巡檢司/ws1ep/未上/ws1ep/以二十九年己卯十一月...,嗚呼，古所謂烈丈夫者，豈爲其氣足以陵三軍，力足以雄萬夫而已哉!拔山扛鼎，用不本於禮誼，則可懾...,享年五十有八
5,5787231.0,董惟昭,董世安,□慶元初蒙慶典恩/ws1ep/受封九品官致仕/ws1ep/享年八十有二/ws1ep/距政和丁...,公姓董氏，諱惟昭，字子□，爲慶元府奉化縣剡源人也。曾大父仲誠、大父珠、父允德。公居家勤儉，行...,享年八十有二
6,5749297.0,陳龜年,陳亮,以淳熙十五年四月癸酉卒于家/ws1ep/得年五十有九/,始余出國北門，彌望沮洳之地而帶以一水，岸行不足以容車馬，湖泊往往隨在而有。舟至松江，風濤汹湧...,得年五十有九
...,...,...,...,...,...,...
2957,4967008.0,周廷構,徐鉉,丙寅歲十月二十二日/ws1ep/終于京師某里之官舍/ws1ep/春秋六十有六/,君諱廷構，字正材，洛陽人也。岐山至德，綿瓜瓞者萬邦；洛宅舊都，守枌榆者百世。簪組相繼，譜諜存...,春秋六十有六
2958,4967079.0,方訥,徐鉉,丙寅歲正月十六日/ws1ep/卒于京師美仁坊官舍/ws1ep/享年七十七/,公諱訥，字希仁。其先河南人也，後世從官，徙籍新安，支派繁衍，遂爲郡之著姓。迨公數世，皆以儒雅...,享年七十七
2959,4966978.0,陶敬宣,徐鉉,春秋五十有二/ws1ep/保大八年夏四月十有八日卒于位/,公諱敬宣，字文褒。其先尋陽人，因官徙籍，今爲合淝人也。西京作相，開國封侯，於是貽孫南國，主盟...,春秋五十有二
2960,4966982.0,賈潭,徐鉉,遘疾還京/ws1ep/保大六年九月二十有一日/ws1ep/卒于江寧永安里官舍/ws1ep/享...,公諱潭，字孟澤，洛陽人也。周先同姓，即列國之諸侯；漢得名臣，乃洛陽之才子。攀鱗河北，豈須方面...,享年六十有八


In [83]:
df_length.to_excel("length.xlsx")

In [46]:
df_length = df_length.set_index('content_id')

In [10]:
# Seperate sentences and retain sentences with kinship words.
all_sent_count=0
kin_sent_count=0
df=pd.DataFrame()
for index in tqdm(df_qsw_raw.index):
    content=df_qsw_raw.loc[index]['content']
    sent_list=sep_mark_sent(content) # Seperate sentences.
    all_sent_count+=len(sent_list)
    subject=df_qsw_raw.loc[index]['subject']
    author=df_qsw_raw.loc[index]['author']
    kin_sent_list=[]
    content_id=index
    for sent in sent_list:
        if "没" in sent or "逝" in sent or "卒" in sent or "薨" in sent or "屬纊" in sent or "終" in sent or "死" in sent or "乃" in sent:
            #if "年" in sent or "月" in sent or "日" in sent:
                sent = "/" + sent + "/"
                kin_sent_list.append(sent)
    df=pd.concat([pd.DataFrame(data=[[content_id, subject, author, sent, content] for sent in kin_sent_list],
                                          columns=['content_id', 'subject', 'author', 'sent', 'content']
                                          ), df], axis=0, ignore_index=True
                            )

100%|██████████| 4701/4701 [00:05<00:00, 925.82it/s] 


In [11]:
# Read kinship dictionary. nk = POS Kinship.
df_nk=pd.read_csv('../data_dict/song_nk_dict.csv', sep='\t', header=None).rename(columns={0:'name', 1:'pos'})
# Read office title. no = office title. noc = office category.
df_no_noc=pd.read_csv('../data_dict/song_no_noc_dict.csv', sep='\t', header=None).rename(columns={0:'name', 1:'pos'})
df_ns=pd.read_csv('../data_dict/song_ns_dict.csv', sep='\t', header=None).rename(columns={0:'name', 1:'pos'})
df_vno=pd.read_csv('../data_dict/song_vno_dict.csv', sep='\t')
df_nz=pd.read_csv('../data_dict/song_nz_dict.csv', sep='\t', header=None).rename(columns={0:'name', 1:'pos'})
df_kin_mismatch=pd.read_csv('../data_dict/kin_mismatch.csv', sep='\t', header=None).rename(columns={0:'name'})

In [12]:
nianhao = pd.read_excel("Nianhao_SongRelated.xlsx")
nianhao

,c_nianhao_id,c_dy,c_dynasty_chn,c_nianhao_chn,c_nianhao_pin,c_firstyear,c_lastyear,Priority
0,387,6,唐,永貞,NaN,805,805,1
1,388,6,唐,元和,NaN,806,820,1
2,389,6,唐,長慶,NaN,821,824,1
3,390,6,唐,寶曆,NaN,825,827,1
4,391,6,唐,大和,NaN,827,835,1
...,...,...,...,...,...,...,...,...
228,634,18,元,元統,NaN,1333,1334,1
229,635,18,元,至元,NaN,1335,1340,1
230,636,18,元,至正,NaN,1341,1370,1
231,637,79,北元,宣光,NaN,1371,1378,1


In [13]:
# Second, subtract place names (POS:ns).
for index in tqdm(df.index):
    sent=df.loc[index]['sent']
    ns_list=[]
    for ns in df_ns['name']:
        if ns in sent:
            ns_list+=re.findall(ns, sent)
            sent=sent.replace(ns, '/ns/')

100%|██████████| 28945/28945 [00:11<00:00, 2592.19it/s]


In [14]:
for index in tqdm(df.index):
    row = df.loc[index]
    nianhao_list = []
    year_list = []
    time_list = []
    for i in nianhao.index:
        hao = nianhao.loc[i]
        if hao['c_nianhao_chn'] in row['sent']:
            re_rule = hao['c_nianhao_chn'] + "[^\/.]*?日" 
            nianhao_list.append(hao['c_nianhao_chn'])
            year_list.append(hao['c_firstyear'])
            l = re.findall(re_rule, row['sent'])
            if l:
                for time in l:
                    time_list.append(time)
            else:
                re_rule = "(" + hao['c_nianhao_chn'] + "[^\/.]*?" + string + ")"
                l = re.findall(re_rule, row['sent'])
                if l:
                    for time in l:
                        time_list.append(time[0])
                else:
                    time_list.append(hao['c_nianhao_chn'])
    if (time_list):
        time_list = list(set(time_list))
        df.loc[index, 'nianhaofull'] = str(time_list)
        df.loc[index, 'nianhao'] = str(nianhao_list)
        df.loc[index, 'time'] = str(year_list)

100%|██████████| 28945/28945 [09:17<00:00, 51.91it/s]


In [20]:
df_die = df.dropna()
df_die.sample(10)

,content_id,subject,author,sent,content,nianhaofull,nianhao,time
23103,5234200.0,司馬京,范祖禹,/長女適左侍禁白昭僅/ws1ep/次早卒/ws1ep/次適大名夏津主簿劉嗣宗/ws1ep/次...,司馬氏出於晉安平獻王孚，其裔曰征東大將軍楊，始居安邑。後魏析安邑置夏縣，子孫遂爲夏縣人。自征...,['乾寧'],['乾寧'],[894]
4648,5502780.0,胡安國,胡寅,/應天尹葉夢得坐爲蔡京所知/ws1ep/落職宫祠/ws1ep/公言/wm/‘京罪已正/ws1...,寶文閣直學士、左朝請郎致仕、南陽縣開國男、食邑三百户、賜紫金魚袋、贈左太中大夫、諡文定胡公行...,['應天'],['應天'],[1206]
23471,5188448.0,趙宗藎,王安禮,/元豐三年二月戊申薨/ws1ep/享年四十有二/,公諱宗藎，字才叔，濮安懿王第二十五子也。起家除右監門率府副率，皇祐中大享明堂，進太子右內率府...,['元豐三年二月戊申'],['元豐'],[1078]
19083,5390728.0,廖倚,陳淵,/以靖康元年十月庚辰終於家/ws1ep/享年九十有四/,公諱某，字成伯，姓廖氏。廖氏之先，蓋出於周文王之子伯廖，以字爲氏。自東漢以來，時有顯者，然皆...,['靖康元年十月庚辰'],['靖康'],[1126]
9769,5889552.0,方大鏞及妻,劉克莊,/寶祐乙卯/ws1ep/年五十七矣/ws1ep/以疾卒于寢/ws1ep/某月某日也/,贈中奉大夫方公逵有三丈夫子：伯揭陽令君大輿〔一〕，仲寶謨閣學士謚忠惠公大琮〔二〕，君其季也，...,['寶祐乙卯'],['寶祐'],[1253]
10255,5870178.0,趙俾夫,袁甫,/尉且更/ws1ep/俄得疾遽卒/ws1ep/享年五十有六/ws1ep/端平元年六月七日也/,余友湯伯紀，端介不妄言，持其所狀婦翁趙君言行示余，曰：‘翁，篤厚人也。儀矩端嚴，行事有典則；...,['端平元年六月七日'],['端平'],[1234]
27429,5062385.0,梅詢,歐陽修,/以疾出知許州/ws1ep/康定二年六月某日/ws1ep/卒於官/,翰林侍讀學士、給事中梅公既卒之明年〔一〕，其孤及其兄之子堯臣來請銘以葬，曰：‘吾叔父病且亟矣...,['康定二年六月某日'],['康定'],[1040]
6586,5131682.0,孫永,蘇頌,/公以元祐二年正月壬午薨於京師〔一〕/ws1ep/嗣子承事郎愭護喪南歸/ws1ep/卜其年五...,有宋四朝耆德之臣，資政殿學士、通議大夫兼侍讀、提舉中太一宫兼集禧觀公事、柱國、樂安郡開國侯，...,['元祐二年正月壬午'],['元祐'],[1086]
28840,8317681.0,錢俶,慎知禮,/端拱元年秋八月二十四日薨于府署/ws1ep/享年六十/,代天之工，必崇高而啓其緒；成物之務，惟光大以垂其式。戴元后，總群牧，開國承家，守宗廟祭祀者，...,['端拱元年秋八月二十四日'],['端拱'],[988]
27185,5067840.0,仁宗妃、錢振女,張方平,/越二年/ws1ep/生皇第二女/ws1ep/號崇慶公主/ws1ep/四歲亦薨/ws1ep/...,國朝憲章古制，叙建内官。上法天極，以正四妃之位；外沿卿職，而修九嬪之教。所以順承后治，變諧陰...,['崇慶'],['崇慶'],[1212]


In [118]:
df_die.iloc[3960]

content_id                                             4966982.0
subject                                                       賈潭
author                                                        徐鉉
sent           /遘疾還京/ws1ep/保大六年九月二十有一日/ws1ep/卒于江寧永安里官舍/ws1ep/...
content        公諱潭，字孟澤，洛陽人也。周先同姓，即列國之諸侯；漢得名臣，乃洛陽之才子。攀鱗河北，豈須方面...
nianhaofull                 ['保大六年九月二十有一日', '保大六年九月二十有一日', '永安']
nianhao                                       ['保大', '保大', '永安']
time                                           [943, 1121, 1099]
Name: 28934, dtype: object

In [21]:
df_die.to_excel("die_doccano.xlsx")

In [20]:
for index in tqdm(df_die.index):
    row = df_die.loc[index]
    nianhao = row['nianhao']
    list = row['sent'].split('/')
    for item in list:
        if nianhao in item:
            df_die.loc[index, 'a'] = item

  0%|          | 0/3962 [00:00<?, ?it/s]/home/jc092/.local/lib/python3.7/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/home/jc092/.local/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
100%|██████████| 3962/3962 [00:01<00:00, 3505.56it/s]


In [21]:
re_rule = "(一|元|二|三|四|五|六|七|八|九|十|十一|十二|十三|十四|十五|十六|十七|十八|十九|二十|二十一|二十二|二十三|二十四|二十五|二十六|二十七|二十八|二十九|三十)年"
digit = {'一': 1, '元': 1, '二': 2, '三': 3, '四': 4, '五': 5, '六': 6, '七': 7, '八': 8, '九': 9, '十':10, '十一':11, '十二':12, '十三':13, '十四':14, '十五' :15, '十六':16, '十七' :17, '十八':18, '十九':19, '二十':20, '二十一':21, '二十二': 22, "二十三":23, "二十四":24, "二十五":25, "二十六" :26, "二十七" :27, "二十八":28,"二十九":29, '三十':30}
for index in tqdm(df_die.index):
    row = df_die.loc[index]
    find = re.findall(re_rule, row['sent'])
    if find:
        df_die.loc[index, 'time'] = digit.get(find[0]) + row['time']

100%|██████████| 3962/3962 [00:00<00:00, 4898.44it/s]


In [22]:
group = df_die.groupby('content_id')
df = pd.DataFrame(columns = ['content_id', 'nianhao', 'time'])
for g in tqdm(group):
    early = 10000
    content_id = g[0]
    for index in g[1].index:
        if df_die.loc[index, 'time'] < early:
            early = df_die.loc[index, 'time']
            nianhao = df_die.loc[index, 'nianhao']
    dict = {'content_id':content_id, 'nianhao': nianhao, 'time': early}
    df = df.append(dict, ignore_index=True)

100%|██████████| 3196/3196 [00:04<00:00, 727.79it/s]


In [23]:
df.sample(20)

,content_id,nianhao,time
2951,5899727.0,咸淳,1267.0
2370,5758234.0,淳熙,1174.0
871,5234417.0,元祐,1094.0
512,5131780.0,元祐,1087.0
2500,5770458.0,嘉定,1208.0
2050,5654569.0,紹熙,1190.0
371,5111159.0,嘉祐,1064.0
381,5111279.0,皇祐,1050.0
2754,5844119.0,紹興,1150.0
2398,5764570.0,慶元,1197.0


In [ ]:
df.to_excel('time.xlsx')

In [ ]:
re_rule = "(.*年.*月.*日).*?\/"
re_rule1 = "\/(?:.(?!\/))+$"
for index in tqdm(df.index):
    row = df.loc[index]
    find = re.findall(re_rule, row['sent'])
    if find:    
        find = re.findall(re_rule1,find[0])
        df.loc[index, 'time'] = find[0].replace("/", "")
        df.loc[index, 'sent'] = df.loc[index, 'sent'].replace(find[0], '/t/')
        

In [ ]:
re_rule = "(.*年.*月).*?\/"
re_rule1 = "\/(?:.(?!\/))+$"
for index in tqdm(df.index):
    row = df.loc[index]
    find = re.findall(re_rule, row['sent'])
    if find:    
        find = re.findall(re_rule1,find[0])
        df.loc[index, 'time'] = find[0].replace("/", "")
        df.loc[index, 'sent'] = df.loc[index, 'sent'].replace(find[0], '/t/')

In [ ]:
df.sample(20)

In [ ]:
df = df.dropna()

In [ ]:
df #无年， 无日 两种特殊情况

In [ ]:
df.sample(20)